# UKV forecast data to interactive web app with annotation tools

## Setup

In [ ]:
import intake
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import cartopy.feature as cf
hv.extension('bokeh')

## Load data using `intake` catalog of Met Office data

In [ ]:
hypercube = intake.cat.mo_aws_earth.ukv.air_pressure_at_sea_level.read()
hypercube

In [ ]:
cube = hypercube[0,0,...]
cube

#### Plot with Matplotlib, as most Met Office scientists will be familiar with.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import iris.quickplot as qplt

# qplt.pcolormesh(cube)
qplt.contourf(cube)
plt.gca().coastlines('50m')
plt.gcf().set_size_inches(12,8)

## Use `geoviews` to create `bokeh` objects

In [ ]:
ds_gv = gv.Dataset(cube, ['projection_x_coordinate','projection_y_coordinate'])

In [ ]:
air_pressure_plot = gv.FilledContours(ds_gv).opts(line_color=None, line_width=0)

In [ ]:
coastlines = gv.feature.coastline
coastlines

In [ ]:
air_pressure_plot.opts(projection=ccrs.Orthographic(0, 30), colorbar=False, global_extent=False) * coastlines

In [ ]:
air_pressure_plot.opts(projection=ccrs.GOOGLE_MERCATOR, global_extent=False, apply_extents=True) * coastlines

## Add `bokeh` annotation tools

In [ ]:
from holoviews.streams import FreehandDraw

warm_front = gv.Path([]).opts(color='red', line_width=9)
warm_front_pen = FreehandDraw(source=warm_front)

warning_orange = gv.Polygons([]).opts(line_color='orange', line_width=9, 
                                      fill_color='orange', fill_alpha=0.6)
warning_orange_tool = FreehandDraw(source=warning_orange)

In [ ]:
plot = air_pressure_plot.opts(projection=ccrs.GOOGLE_MERCATOR, global_extent=False) * coastlines * warm_front * warning_orange

In [ ]:
plot.opts(width=800, height=720)

## Serve up plot as an app

In [ ]:
def make_annotable(plot, port=0, websocket_origin='pangeo.informaticslab.co.uk', url_path='annotable'):
    import holoviews as hv
    from bokeh.server.server import Server
    import os
    from IPython.core.display import display, HTML
    import ipywidgets as widgets
    
    renderer = hv.renderer('bokeh')
    app = renderer.app(plot)
    server = Server({f'/{url_path}': app}, port=port, allow_websocket_origin=[websocket_origin])
    
    server.start()
    
    prefix = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    url = f"https://{websocket_origin}{prefix}proxy/{server.port}/{url_path}"
    display(HTML(f'<a href={url}>{url}</a>'))
    
    stop_button = widgets.Button(description="Stop Annotable")
    stop_button.on_click(lambda b: server.stop())
    display(stop_button)
    
    return server

In [ ]:
annotable = make_annotable(plot)

#### Access annotations

In [ ]:
warm_front_pen.element

In [ ]:
warning_orange_tool.element